In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from importlib import reload
import json
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import MultiLearn_GLRM

In [19]:
df_train = pd.read_csv('./microarray/xtrain.data', sep='\s+', header=None)
df_label_train = pd.read_csv('./microarray/ytrain.data', sep='\s+', header=None)
df_test = pd.read_csv('./microarray/xtest.data', sep='\s+', header=None)
df_label_test = pd.read_csv('./microarray/ytest.data', sep='\s+', header=None)

df_train.loc[-1] = df_label_train.loc[0].to_list()
df_train.index = df_train.index + 1  
df_train = df_train.sort_index()
df_train = df_train.T

df_test.loc[-1] = df_label_test.loc[0].to_list()
df_test.index = df_test.index + 1  
df_test = df_test.sort_index()
df_test = df_test.T
df_test = df_test.dropna()

X_train = df_train.drop(columns=[0])
Y_train = df_train[0]
Y_train = np.array(Y_train, dtype=np.int64).reshape(-1,1)

X_test = df_test.drop(columns=[0])
Y_test = df_test[0]
Y_test = np.array(Y_test, dtype=np.int64).reshape(-1,1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


params = {
    "train_hyper_params": None,
    "test_hyper_params": None,
    "beta": None,
    "embedding_matrix": None,
    "train_error": None,
    "test_error": None,
    "embedding_dim": None,
    "predictor_scaling_parmas": None,
    "target_scaling_parmas": None
}

In [21]:
params["predictor_scaling_parmas"] =  {
    "mean" : scaler.mean_.tolist(),
    "var" : scaler.mean_.tolist()
}

In [20]:
n_class = max(max(Y_train), max(Y_test))[0]

one_hot_targets = np.eye(n_class)[Y_train.reshape(-1,)-1]
Train = np.hstack((X_train, one_hot_targets))
np.random.shuffle(Train)

In [26]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.supervised_embedding_params import Multilearn_GLRM_Supervised_Embeddings_Train_Params, Multilearn_GLRM_Supervised_Embeddings_Test_Params

embedding_dim = 20
functional_loss='WLSE'
params["embedding_dim"] = embedding_dim

n_class = n_class

A_prime, X_prime, Y_prime = generate_AXY.get_supervised_embedding_glrm_train_form(Train[:,:-n_class], Train[:,-n_class:], n_class, embedding_dim)
GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions = Multilearn_GLRM_Supervised_Embeddings_Train_Params(A_prime, X_prime, Y_prime, n_class)

params["train_hyper_params"] = [GLRM_loss_list, X_regulariation_list, Y_regulariation_list]

num_iterations= 200
learning_rate=0.05
result = Multi_Learn.alternating_minimization(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions, num_iterations, learning_rate, n_class, functional_loss=functional_loss)

epoch: 0
iter: 1 Total loss: tf.Tensor(23.219868052901376, shape=(), dtype=float64) [4.595402075471945, 20.031861142819945]
iter: 67 Total loss: tf.Tensor(11.618460227990836, shape=(), dtype=float64)
iter: 1 Total loss: tf.Tensor(11.613192596348481, shape=(), dtype=float64)
iter: 47 Total loss: tf.Tensor(0.9770960138407593, shape=(), dtype=float64)
epoch: 1
iter: 1 Total loss: tf.Tensor(0.9766250150246143, shape=(), dtype=float64) [0.5965622271432717, 0.9895781264457892]
iter: 29 Total loss: tf.Tensor(0.7082633597281974, shape=(), dtype=float64)
iter: 1 Total loss: tf.Tensor(0.7079515726999297, shape=(), dtype=float64)
iter: 31 Total loss: tf.Tensor(0.6489566486474923, shape=(), dtype=float64)
epoch: 2
iter: 1 Total loss: tf.Tensor(0.6487903546737517, shape=(), dtype=float64) [0.0809762515520296, 0.8742002196542202]
iter: 14 Total loss: tf.Tensor(0.6272095078541872, shape=(), dtype=float64)
iter: 1 Total loss: tf.Tensor(0.6269702545439384, shape=(), dtype=float64)
iter: 7 Total loss: t

In [29]:
beta = result[1][1:,0:n_class]
Y_final = result[1][1:embedding_dim+1,n_class:]
params["beta"] = beta.numpy().tolist()
params["embedding_matrix"] = Y_final.numpy().tolist()

train_prob = tf.nn.softmax(np.matmul(result[0], result[1][:,0:n_class])).numpy()
label_train_shuffled =  np.argmax(Train[:,-n_class:], axis=1)
params["train_accuracy"] = accuracy_score(label_train_shuffled, np.argmax(train_prob, axis=1))
params["train_accuracy"]

1.0

In [28]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.supervised_embedding_params import Multilearn_GLRM_Supervised_Embeddings_Train_Params, Multilearn_GLRM_Supervised_Embeddings_Test_Params

A = X_test
A_prime =tf.constant(A)


X = np.random.standard_normal((A.shape[0], embedding_dim))
X_prime = X
X_prime = tf.Variable(X_prime)

Y_prime = Y_final

one_hot_targets_val = np.eye(n_class)[Y_test.reshape(-1,)-1]
one_hot_targets_val

GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, X_grad_restrictions, Y_grad_restrictions =  Multilearn_GLRM_Supervised_Embeddings_Test_Params(A_prime, X_prime, Y_prime)
params["test_hyper_params"] = [GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list]

num_iterations=10000
learning_rate=0.1
results_log = []
for i in range(0,20):
    A = X_test
    A_prime =tf.constant(A)


    X = np.random.standard_normal((A.shape[0], embedding_dim))
    X_prime = X
    X_prime = tf.Variable(X_prime)

    Y_prime = Y_final

    result_val = Multi_Learn.predict(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, num_iterations, learning_rate)
    val_pred = result_val[0]
    val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
    val_pred = np.matmul(val_pred, beta)
    val_pred = tf.nn.softmax(val_pred).numpy()
    accuracy = accuracy_score(Y_test-1, np.argmax(val_pred, axis=1))

    print(accuracy)
    results_log.append(accuracy)


params["accuracy"] = results_log
1 - np.mean(results_log)

0.95
0.95
1.0
0.95
0.95
0.95
1.0
1.0
0.95
1.0
0.95
0.95
0.95
0.95
0.95
0.95
0.95
0.95
0.95
0.95


0.04000000000000026

In [ ]:
beta = result[1][1:,0:n_class]
Y_final = result[1][1:embedding_dim+1,n_class:]
params["beta"] = beta.numpy().tolist()
params["embedding_matrix"] = Y_final.numpy().tolist()

train_prob = tf.nn.softmax(np.matmul(result[0], result[1][:,0:n_class])).numpy()
label_train_shuffled =  np.argmax(Train[:,-n_class:], axis=1)
params["train_accuracy"] = accuracy_score(label_train_shuffled, np.argmax(train_prob, axis=1))
params["train_accuracy"]

In [ ]:
label_train_shuffled =  np.argmax(Train[:,-n_class:], axis=1)
label_train_shuffled

In [ ]:
np.argmax(train_prob, axis=1)

In [ ]:
np.mean(beta, axis=1)

In [ ]:
np.mean(train_prob, axis=0)

In [ ]:
# dict_keys(['SupervisedEmbeddings', 'Factorization', 'WLSE'])
# result[-1].keys()
len(result[-1]['SupervisedEmbeddings']), len(result[-1]['Factorization'])

In [ ]:
x_values = list(range(0, len(result[-1]['SupervisedEmbeddings'])))
len(x_values[::20])

In [ ]:
plt.figure(figsize=(10, 6))
x_values = list(range(0, len(result[-1]['SupervisedEmbeddings'])))

plt.plot(x_values[::20], result[-1]['SupervisedEmbeddings'][::20],'-.', label="Supervised Learning (w: 0.5)")
plt.plot(x_values[::20], result[-1]['Factorization'][::20], '-.', label="Matrix Factorization (w: 0.5)")
plt.plot(x_values[::20], result[-1]['weighted_average'][::20],'-x', label="Functional: Weighted Average")
plt.yscale('log')
# plt.xticks( x_values[::20])
plt.ylabel('Loss (Log Scale)')
plt.xlabel('Iteration')
plt.title('Task Loss vs Iteration (alpha=0.1)')
plt.legend()
plt.savefig('./microarray/plots/MSE_wavg_microarray_loss_vs_iteration_alpha_1e-1')

In [ ]:
# alpha = 0.1
# w1 = 0.5
# w2= 0.5

# SupervisedEmbeddings_weights = w1*np.exp(alpha*np.array(result[-1]['SupervisedEmbeddings']))
# Factorization_weights = w2*np.exp(alpha*np.array(result[-1]['Factorization']))
# SupervisedEmbeddings_weights_normalized = SupervisedEmbeddings_weights/(SupervisedEmbeddings_weights+Factorization_weights)
# Factorization_weights_normalized = Factorization_weights/(SupervisedEmbeddings_weights+Factorization_weights)

# plt.figure(figsize=(10, 6))
# x_values = list(range(0, len(result[-1]['SupervisedEmbeddings'])))

# plt.plot(x_values[::20], SupervisedEmbeddings_weights_normalized[::20],'-s', label="Supervised Learning")
# plt.plot(x_values[::20], Factorization_weights_normalized[::20], '-s', label="Matrix Factorization")

# # plt.xticks( x_values[::20])
# plt.ylabel('Task Weights')
# plt.xlabel('Iteration')
# plt.title('Task Weights vs Iteration (alpha=0.1)')
# plt.legend()

# plt.savefig('./microarray/plots/MSE_max_microarray_task_weight_vs_iteration_alpha_1e-1')


In [ ]:
# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4495674496068231

# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4011543372347976


In [ ]:
A_prime.shape

In [ ]:
"""
<tf.Tensor: shape=(11, 1), dtype=float64, numpy=
array([[-0.02956712],
       [-0.43500185],
       [ 0.46663761],
       [ 0.28479513],
       [-0.04591487],
       [ 0.22732162],
       [ 0.19878154],
       [-0.21300145],
       [ 0.0071286 ],
       [ 0.11102323],
       [-0.00936144]])>
"""

In [ ]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.supervised_embedding_params import Multilearn_GLRM_Supervised_Embeddings_Train_Params, Multilearn_GLRM_Supervised_Embeddings_Test_Params

A = X_test
A_prime =tf.constant(A)


X = np.random.standard_normal((A.shape[0], embedding_dim))
X_prime = X
X_prime = tf.Variable(X_prime)

Y_prime = Y_final

one_hot_targets_val = np.eye(n_class)[Y_test.reshape(-1,)-1]
one_hot_targets_val

GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, X_grad_restrictions, Y_grad_restrictions =  Multilearn_GLRM_Supervised_Embeddings_Test_Params(A_prime, X_prime, Y_prime)
params["test_hyper_params"] = [GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list]

num_iterations=10000
learning_rate=0.1
results_log = []
for i in range(0,20):
    A = X_test
    A_prime =tf.constant(A)


    X = np.random.standard_normal((A.shape[0], embedding_dim))
    X_prime = X
    X_prime = tf.Variable(X_prime)

    Y_prime = Y_final

    result_val = Multi_Learn.predict(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, num_iterations, learning_rate)
    val_pred = result_val[0]
    val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
    val_pred = np.matmul(val_pred, beta)
    val_pred = tf.nn.softmax(val_pred).numpy()
    accuracy = accuracy_score(Y_test-1, np.argmax(val_pred, axis=1))

    print(accuracy)
    results_log.append(accuracy)


params["accuracy"] = results_log

In [ ]:
params['train_hyper_params'] = []

with open('./logs/Microarray_'+ str(round(np.mean(results_log),5))+'_Accuracy_MSE_MAX.json', 'w') as f:
    f.write(json.dumps(params, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
Y_final.shape

In [ ]:
pd.DataFrame(Y_final.numpy()).to_csv('gene_embedding_data.csv', index=False)

In [ ]:
X_train.shape

In [ ]:
Y_test-1

In [ ]:
np.argmax(val_pred, axis=1)

In [ ]:
params['tol'] = 10e-6
# type(params['embedding_matrix'])
for key in params.keys():
    print(key, type(params[key]))

In [ ]:
# np.argmax(val_pred, axis=1)
# val_pred
# len(Y_test-1)
Y_test-1

In [ ]:
np.argmax(val_pred, axis=1)

In [ ]:
params

In [ ]:
val_pred = np.hstack((result_val[0], A_prime))
val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
regress_val = np.matmul(val_pred, beta)
np.mean((scaler2.inverse_transform(Y_test) - scaler2.inverse_transform(regress_val))**2)

In [ ]:
regress_val

In [ ]:
Y_train

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, predictions_val), confusion_matrix(Y_train, predictions)

In [ ]:
409/(409+200), 409/(409+857)

In [ ]:
305/(305+133), 305/(305+961)

In [ ]:
2027/(2027+538), 2027/(2027+3343)

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_embedding = result[0][:, 1:embedding_dim+1]
X_train_embedding = X_train_embedding.numpy()

X_test_embedding = result_val[0].numpy()


clf = LogisticRegression().fit(X_train_embedding, Y_train)
pred_logistic = clf.predict(X_train_embedding)
pred_logistic_val = clf.predict(X_test_embedding)
accuracy_score(Y_train, pred_logistic), accuracy_score(Y_test, pred_logistic_val)

In [ ]:
confusion_matrix(Y_test, pred_logistic_val), confusion_matrix(Y_train, pred_logistic)

In [ ]:
328/(328+236), 328/(328+938)

In [ ]:
1565/(1565+650), 1565/(1565+3805)

In [ ]:
beta[5:]

In [ ]:
A_prime[0]

In [ ]:
x_embeddings

In [ ]:
prob_val[0:10]

In [ ]:
prob[0:10]

In [ ]:
temp = tf.Variable([1,1,1])
tf.where(temp!=1).shape[0]
# tf.where([True, False, False, True]).numpy()

In [ ]:
np.min(val_pred,axis=0)

In [ ]:
np.min(result[0],axis=0)

In [ ]:
beta

In [ ]:
result[0][3:10,:]

In [ ]:
'beta' in params.keys()